## Bike Sharing Demand

* 시내 키오스크 네트워크를 통해 자전거 대여가 이루워짐.
* 이 시스템을 통해 생성된 데이터를 사용

#### 참조
* https://www.kaggle.com/code/viveksrinivasan/eda-ensemble-model-top-10-percentile
* https://www.kaggle.com/code/kwonyoung234/for-beginner

### 진행 방향
* About Dataset
* Data Summary
* Feature Engineering
* Missing Value Analysis
* Outlier Analysis

### 1. About Dataset

#### **Data Fields**
* datetime - hourly date + timestamp  
* season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weather 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain +Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* count - number of total rentals

In [ ]:
# 라이브러리 import

import calendar
import numpy as np
import pandas as pd
import seaborn as sn
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 데이터 불러오기
train = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
# 훈련 데이터의 모형 파악하기
train.head()

In [ ]:
# 데이터에 대한 전반적인 정보 확인.
train.info()

In [ ]:
# 데이터 타입만 보기
train.dtypes

### 2. Feature Engineering

* "datetime" 열에서 "date", "hour", "weekDay", "month" 열을 새로 만들기
* "date", "hour", "weekDay", "month" 열을 숫자형 데이터로 바꾸기
* "datetime" 열에서 유용하게 추출했으니 해당 열을 drop하기

#### **코드 쓰기 전 필요한 개념들**

#### 1) datetime
* 내장 모듈인 datetime, 날짜, 시간을 저장하는 datetime 객체를 반환
* 패키지 이름과 클래스 이름이 같으니 주의.

#### 2) apply

* DataFrame의 칼럼에 복잡한 연산을 vectorizing할 수 있게 해주는 함수로 자주 활용.

#### 3) lambda

* 한줄(일시적) 함수, lambda x, y : x + y 형태

#### 4) split

* 문자열 자르기

#### **1) "datetime" 열에서 "date", "hour", "weekDay", "month" 열을 새로 만들기**

In [ ]:
# datetime이 어떻게 생겼는지 살펴보기
train.datetime[0]

In [ ]:
#split함수로 년-월-일 과 시간을 분리 -> tempDate에 저장
train['tempDate'] = train.datetime.apply(lambda x:x.split())

* weekday는 calendar패키지와 datetime패키지를 활용한다.
* calendar.day_name 사용법 : https://stackoverflow.com/questions/36341484/get-day-name-from-weekday-int
* datetime.strptime 문서: https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior

In [ ]:
#분리한 tempDate -> year,month,day,weekday column 추출
train['year'] = train.tempDate.apply(lambda x:x[0].split('-')[0])
train['month'] = train.tempDate.apply(lambda x:x[0].split('-')[1])
train['day'] = train.tempDate.apply(lambda x:x[0].split('-')[2])
train['weekday'] = train.tempDate.apply(lambda x:calendar.day_name[datetime.strptime(x[0],"%Y-%m-%d").weekday()])
train['hour'] = train.tempDate.apply(lambda x:x[1].split(':')[0])

#### **2) "date", "hour", "weekDay", "month" 열을 숫자형 데이터로 바꾸기**

* pandas.to_numeric(): https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html

In [ ]:
#분리를 통해 추출된 속성은 문자열 속성을 가지고 있음 따라서 숫자형 데이터로 변환해 줄 필요가 있음.
train['year'] = pd.to_numeric(train.year,errors='coerce')
train['month'] = pd.to_numeric(train.month,errors='coerce')
train['day'] = pd.to_numeric(train.day,errors='coerce')
train['hour'] = pd.to_numeric(train.hour,errors='coerce')

#### **3) "datetime" 열에서 유용하게 추출했으니 해당 열을 drop하기**

In [ ]:
#필요를 다한 tempDate column을 drop함
train = train.drop('tempDate',axis=1)

In [ ]:
train.head()

#### **Option) Variables DataType Count 시각화**

In [ ]:
#각각의 속성과 예측의 결과값으로 쓰이는 count값과의 관계 파악

#년도와 count
fig = plt.figure(figsize=[12,10])
ax1 = fig.add_subplot(2,2,1)
ax1 = sn.barplot(x='year',y='count',data=train.groupby('year')['count'].mean().reset_index())

#month와 count
ax2 = fig.add_subplot(2,2,2)
ax2 = sn.barplot(x='month',y='count',data=train.groupby('month')['count'].mean().reset_index())

#day와 count
ax3 = fig.add_subplot(2,2,3)
ax3 = sn.barplot(x='day',y='count',data=train.groupby('day')['count'].mean().reset_index())

#hour와 count
ax4 = fig.add_subplot(2,2,4)
ax4 = sn.barplot(x='hour',y='count',data=train.groupby('hour')['count'].mean().reset_index())

In [ ]:
#계절과 count
fig = plt.figure(figsize=[12,10])
ax1 = fig.add_subplot(2,2,1)
ax1 = sn.barplot(x='season',y='count',data=train.groupby('season')['count'].mean().reset_index())

#휴일 여부와 count
ax2 = fig.add_subplot(2,2,2)
ax2 = sn.barplot(x='holiday',y='count',data=train.groupby('holiday')['count'].mean().reset_index())

#작업일 여부와 count
ax3 = fig.add_subplot(2,2,3)
ax3 = sn.barplot(x='workingday',y='count',data=train.groupby('workingday')['count'].mean().reset_index())

#날씨와 count
ax4 = fig.add_subplot(2,2,4)
ax4 = sn.barplot(x='weather',y='count',data=train.groupby('weather')['count'].mean().reset_index())

### 3. Missing Values Analysis

* 데이터와 열을 파악한 후 다음 단계는 데이터에 결측값이 있는지 확인하기
* 결측값을 확인하는 방법: 1) isnull 2) "missingno" 라이브러리 활용하기

#### 1) isnull로 결측값 확인하기

In [ ]:
train.isnull().sum() # isnull = isna

#### 2) "missingno" 라이브러리 활용하기

In [ ]:
msno.matrix(train,figsize=(12,5))

### 4. Outliers Analysis

* "count" 변수에는 이상치가 존재. -> 시각화를 통해 확인

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sn.boxplot(data=train,y="count",orient="v",ax=axes[0][0])
sn.boxplot(data=train,y="count",x="season",orient="v",ax=axes[0][1])
sn.boxplot(data=train,y="count",x="hour",orient="v",ax=axes[1][0])
sn.boxplot(data=train,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

#### count 열에서 이상치 제거하기

##### numpy 관련 함수
* abs() : 절대값 계산
* mean() : 산술 평균 계산
* std() : 표준 편차 계산

In [ ]:
trainWithtoutOutlier = train[np.abs(train["count"]-train["count"].mean()) <=(3*train["count"].std())]

In [ ]:
print ("Shape Of The Before Ouliers: ",train.shape)
print ("Shape Of The After Ouliers: ",trainWithtoutOutlier.shape)

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sn.boxplot(data=trainWithtoutOutlier,y="count",orient="v",ax=axes[0][0])
sn.boxplot(data=trainWithtoutOutlier,y="count",x="season",orient="v",ax=axes[0][1])
sn.boxplot(data=trainWithtoutOutlier,y="count",x="hour",orient="v",ax=axes[1][0])
sn.boxplot(data=trainWithtoutOutlier,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")